# Check python version, conda environment and GPU

In [1]:
!python -V
# !pip install -U click
# !pip install -U pandas
# %pip install -U so-vits-svc-fork
# %pip install -U ipython

Python 3.10.13


In [2]:
!conda env list

# conda environments:
#
base                     /home/jorge/miniconda3
prueba                   /home/jorge/miniconda3/envs/prueba
so-vits-fork          *  /home/jorge/miniconda3/envs/so-vits-fork



In [3]:
!nvidia-smi

Tue Mar 26 11:50:25 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:68:00.0 Off |                  N/A |
| 30%   31C    P8    27W / 350W |     19MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Create auxiliary functions

In [4]:
import shutil
import glob
import random
import os

def create_dataset_raw(speaker_id):

    destination_directory = os.path.join('dataset_raw', speaker_id)

    os.makedirs(destination_directory)
    
    wav_files = glob.glob(os.path.join('OriginalAudios', speaker_id + '/**/*.wav'), recursive=True)
    
    for wav_file in wav_files:
      shutil.copy(wav_file, destination_directory)

In [5]:
import pandas as pd
import json

def preprocessing(speaker_id, epochs=500, eval_interval_epochs=50, log_interval_epochs=25):
         
    !svc pre-resample
    !svc pre-config
    
    # Calculate the steps per epoch, i.e. the steps to go through all the samples
    config = pd.read_json('configs/44k/config.json')
    batch_size = config.loc['batch_size', 'train']
    n_training_samples = len(os.listdir(os.path.join('dataset_raw', speaker_id)))
    steps_per_epoch = n_training_samples / batch_size

    # Calculate the recommended eval interval and log_interval
    eval_interval = eval_interval_epochs * (int(steps_per_epoch) + 1)
    log_interval = log_interval_epochs * (int(steps_per_epoch) + 1) 

    # Read the JSON file
    with open('configs/44k/config.json', 'r') as file:
        data = json.load(file)
    
    # Modify the desired field
    data['train']['log_interval'] = log_interval
    data['train']['eval_interval'] = eval_interval
    data['train']['epochs'] = epochs
    
    # Write the modified data back to the JSON file
    with open('configs/44k/config.json', 'w') as file:
        json.dump(data, file, indent=4)

    !svc pre-hubert

In [6]:
def training(speaker_id):
    !svc train --model-path logs/{speaker_id}

In [7]:
def delete_created_folders():
    shutil.rmtree('configs')
    shutil.rmtree('dataset')
    shutil.rmtree('dataset_raw')
    shutil.rmtree('filelists')

# Main

In [8]:
def main():
    speaker_ids = ['8699']
    for speaker_id in speaker_ids:
        create_dataset_raw(speaker_id)
        preprocessing(speaker_id, epochs=2000, eval_interval_epochs=400, log_interval_epochs=400)
        training(speaker_id)
        delete_created_folders()
        

# Run

In [ ]:
main()

  0%|                                                   | 0/791 [00:00<?, ?it/s][11:50:38] WARNING  [11:50:38]                                   ]8;id=805750;file:///home/jorge/miniconda3/envs/so-vits-fork/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=564438;file:///home/jorge/miniconda3/envs/so-vits-fork/lib/python3.10/warnings.py#109\109]8;;\
                    /home/jorge/miniconda3/envs/so-vits-fork/lib                
                    /python3.10/site-packages/so_vits_svc_fork/p                
                    reprocessing/preprocess_flist_config.py:41:                 
                    argument 'filename' has been renamed to                     
                    'path' in version 0.10.0.                                   
                            This alias will be removed in                       
                    version 1.0.                                                
                      if get_duration(filename=path) < 0.3:               